In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=595c2b78ffd2a20ba5f2ee67a50f2d6718df57587add82b135a5409d6d05e3d8
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [33]:
import zipfile
import os
import pandas as pd
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import Window
from datetime import datetime

In [4]:
spark = SparkSession.builder.appName("Exercise6").getOrCreate()
print(spark)

In [18]:
def read_csv_from_zip():
  # the cols in 2019_Q4 & 2020_Q1 are different so can't be merged
  # only using the former csv
  download_loc = '/content/'
  pandas_df = pd.DataFrame()
  for item in os.listdir(download_loc):
    if item.endswith('.zip'):
      file_name = os.path.abspath(item)
      print("File name: ", file_name)
      zip_ref = zipfile.ZipFile(file_name)
      for f in zip_ref.namelist():
        if f.endswith('csv'):
          with zip_ref.open(f) as f:
              pandas_df = pd.read_csv(f, encoding_errors='ignore')
              df = spark.createDataFrame(pandas_df)
              return df

In [19]:
df = read_csv_from_zip()
df.show()

File name:  /content/Divvy_Trips_2019_Q4.zip
+--------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
| trip_id|         start_time|           end_time|bikeid|tripduration|from_station_id|   from_station_name|to_station_id|     to_station_name|  usertype|gender|birthyear|
+--------+-------------------+-------------------+------+------------+---------------+--------------------+-------------+--------------------+----------+------+---------+
|25223640|2019-10-01 00:01:39|2019-10-01 00:17:20|  2215|       940.0|             20|Sheffield Ave & K...|          309|Leavitt St & Armi...|Subscriber|  Male|   1987.0|
|25223641|2019-10-01 00:02:16|2019-10-01 00:06:34|  6328|       258.0|             19|Throop (Loomis) S...|          241| Morgan St & Polk St|Subscriber|  Male|   1998.0|
|25223642|2019-10-01 00:04:32|2019-10-01 00:18:43|  3003|       850.0|             84|Milwaukee Ave 

Bring data into proper format.

In [20]:
df = df.withColumn('start_time', unix_timestamp('start_time', 'yyyy-MM-dd HH:mm:ss').cast('timestamp')) \
.withColumn('end_time', unix_timestamp('end_time', 'yyyy-MM-dd HH:mm:ss').cast('timestamp')) \
.withColumn('tripduration', col('tripduration').cast('long'))

In [21]:
df.printSchema()

root
 |-- trip_id: long (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- bikeid: long (nullable = true)
 |-- tripduration: long (nullable = true)
 |-- from_station_id: long (nullable = true)
 |-- from_station_name: string (nullable = true)
 |-- to_station_id: long (nullable = true)
 |-- to_station_name: string (nullable = true)
 |-- usertype: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birthyear: double (nullable = true)



What is the average trip duration per day?


In [22]:
def avg_trip_duration_per_day(df):
  average_trip_duration_per_day = df.groupBy(df.start_time.cast('date').alias('date')).agg(avg('tripduration').alias('average_trip_duration_seconds'))

  return average_trip_duration_per_day.sort("date").show()

avg_trip_duration_per_day(df)

+----------+-----------------------------+
|      date|average_trip_duration_seconds|
+----------+-----------------------------+
|2019-10-01|            519.1034563470391|
|2019-10-02|            489.1325637447672|
|2019-10-03|            507.1099269445638|
|2019-10-04|            502.3697794462694|
|2019-10-05|            529.6975832789027|
|2019-10-06|            540.2795057520239|
|2019-10-07|            515.5357023690357|
|2019-10-08|            515.4543694020819|
|2019-10-09|            512.1497682738434|
|2019-10-10|           505.07017693819984|
|2019-10-11|            475.8004881025015|
|2019-10-12|            500.3075485799701|
|2019-10-13|            522.7091085584857|
|2019-10-14|           497.98104315039797|
|2019-10-15|           495.01938976377954|
|2019-10-16|            489.1441823744202|
|2019-10-17|             492.546112804878|
|2019-10-18|           501.65576186265844|
|2019-10-19|            543.4861073644456|
|2019-10-20|            519.4521739130435|
+----------

How many trips were taken each day?


In [23]:
def trips_per_day(df):
  df = df.withColumn('start_date', to_date('start_time'))
  trips_per_day = df.groupBy('start_date').agg(count('*').alias('trips_count'))
  return trips_per_day.sort("start_date").show()

trips_per_day(df)

+----------+-----------+
|start_date|trips_count|
+----------+-----------+
|2019-10-01|      18425|
|2019-10-02|       9882|
|2019-10-03|      15647|
|2019-10-04|      14570|
|2019-10-05|      10452|
|2019-10-06|      13396|
|2019-10-07|      17256|
|2019-10-08|      17537|
|2019-10-09|      17226|
|2019-10-10|      15795|
|2019-10-11|       8016|
|2019-10-12|       8702|
|2019-10-13|      10533|
|2019-10-14|      13785|
|2019-10-15|      13297|
|2019-10-16|      12886|
|2019-10-17|      13635|
|2019-10-18|      14096|
|2019-10-19|      13953|
|2019-10-20|      10419|
+----------+-----------+
only showing top 20 rows



What was the most popular starting trip station for each month?

In [24]:
def popular_station_monthwise(df):
  df = df.withColumn('month', month('start_time'))
  station_counts = df.groupBy('month', 'from_station_name').agg(count('*').alias('station_count'))

  window_spec = Window.partitionBy('month').orderBy(desc('station_count'))
  ranked_stations = station_counts.withColumn('rank', rank().over(window_spec))
  most_popular_station_per_month = ranked_stations.filter(col('rank') == 1) \
    .select('month', 'from_station_name', 'station_count').orderBy('month')
  return most_popular_station_per_month.show()

popular_station_monthwise(df)

+-----+-------------------+-------------+
|month|  from_station_name|station_count|
+-----+-------------------+-------------+
|   10|Canal St & Adams St|         6564|
|   11|Canal St & Adams St|         3445|
|   12|Canal St & Adams St|         2928|
+-----+-------------------+-------------+



What were the top 3 trip stations each day for the last two weeks?

In [27]:
def top3(df):
  df = df.withColumn('start_date', to_date('start_time'))
  max_date = df.select(max('start_date')).collect()[0][0]

  df_last_two_weeks = df.filter(col('start_date') >= (max_date - expr('INTERVAL 14 DAYS')))

  station_counts = df_last_two_weeks.groupBy('start_date', 'from_station_name').agg(count('*').alias('station_count'))

  # Rank stations within each day based on count
  window_spec = Window.partitionBy('start_date').orderBy(col('station_count').desc())

  ranked_stations = station_counts.withColumn('rank', rank().over(window_spec))

  # Filter to keep only the top 3 stations for each day
  top_3_trip_stations_per_day = ranked_stations.filter(col('rank') <= 3).select('start_date', 'from_station_name', 'station_count').orderBy('start_date', 'rank')
  return top_3_trip_stations_per_day.show()


top3(df)

+----------+--------------------+-------------+
|start_date|   from_station_name|station_count|
+----------+--------------------+-------------+
|2019-12-17| Canal St & Adams St|          153|
|2019-12-17|Clinton St & Madi...|          144|
|2019-12-17|Clinton St & Wash...|          124|
|2019-12-18| Canal St & Adams St|          123|
|2019-12-18|Clinton St & Madi...|          115|
|2019-12-18|Clinton St & Wash...|           94|
|2019-12-19| Canal St & Adams St|          133|
|2019-12-19|Clinton St & Madi...|          123|
|2019-12-19|Clinton St & Wash...|           95|
|2019-12-20| Canal St & Adams St|          131|
|2019-12-20|Clinton St & Wash...|          109|
|2019-12-20|Clinton St & Madi...|           94|
|2019-12-21|Streeter Dr & Gra...|           63|
|2019-12-21|Kingsbury St & Ki...|           47|
|2019-12-21|Wells St & Concor...|           46|
|2019-12-22|      Shedd Aquarium|           87|
|2019-12-22|Lake Shore Dr & M...|           79|
|2019-12-22|Streeter Dr & Gra...|       

Do Males or Females take longer trips on average?

In [31]:
df.groupBy('gender').agg(avg('tripduration').alias('genderwise_trip_dur')).show()

+------+-------------------+
|gender|genderwise_trip_dur|
+------+-------------------+
|Female|  509.8082474784837|
|   NaN|  611.8951609483015|
|  Male|  478.6205057415161|
+------+-------------------+



What is the top 10 ages of those that take the longest trips, and shortest?

In [44]:
def top_10_longest(df):
  df = df.withColumn('age', year('start_time') - col('birthyear'))

  # Group by age and sum the trip durations
  age_trip_durations = df.groupBy('age').agg({'trip_duration_seconds': 'sum'})

  # Order by total trip duration in descending order
  top_10_longest_trip_ages = age_trip_durations.orderBy(desc('sum(trip_duration_seconds)')).limit(10)

  return top_10_longest_trip_ages.show()

print("Top 10 ages of those who take the longest trips:")
top_10_longest(df)

Top 10 ages of those who take the longest trips:
+----+--------------------------+
| age|sum(trip_duration_seconds)|
+----+--------------------------+
| NaN|                 245822730|
|27.0|                  33720565|
|28.0|                  33632688|
|25.0|                  32515421|
|29.0|                  32368795|
|26.0|                  29348024|
|30.0|                  28928675|
|32.0|                  28793473|
|31.0|                  26060245|
|33.0|                  23483987|
+----+--------------------------+

